In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import pandas as pd
HERE = %pwd
sys.path.append(os.path.dirname(HERE))

In [2]:
from src import utils

utils.set_seed()

# if you have another LLM, please modify src/llm.py and src/utils.py
model_name = "gpt-4o-mini-2024-07-18"
llm = utils.load_llm(model_name)

In [3]:
# prompt example
messages = [
    {"role" : "system", "content" : "You speak Italian language"},
    {"role" : "user", "content" : "Please tell me some fruit name."}
]

# generated text + log info
output, dict_log = llm(messages, log=True)

In [4]:
output

'Certo! Ecco alcuni nomi di frutta:\n\n1. Mela\n2. Banana\n3. Arancia\n4. Pera\n5. Uva\n6. Fragola\n7. Kiwi\n8. Pesca\n9. Ananas\n10. Ciliegia\n\nSe hai bisogno di ulteriori informazioni su qualche frutto in particolare, fammelo sapere!'

In [5]:
dict_log

{'time': 1.8111209869384766,
 'input token': 22,
 'output token': 81,
 'input text': [{'role': 'system', 'content': 'You speak Italian language'},
  {'role': 'user', 'content': 'Please tell me some fruit name.'}],
 'output text': 'Certo! Ecco alcuni nomi di frutta:\n\n1. Mela\n2. Banana\n3. Arancia\n4. Pera\n5. Uva\n6. Fragola\n7. Kiwi\n8. Pesca\n9. Ananas\n10. Ciliegia\n\nSe hai bisogno di ulteriori informazioni su qualche frutto in particolare, fammelo sapere!'}

In [6]:
data_name = "Amazon_Movie"
exp_name = "light"
version_input = "20250403_input"
version_prep = "20250403_prep"
n_user = 5

# load input data
from src import data_loader as dl
dict_data = dl.load_data(
    data_name=data_name,
    exp_name=exp_name,
    model_name=model_name,
    version_input=version_input,
    version_prep=version_prep,
    n_user=n_user
)     

# single user data
users = list(dict_data.keys())
user = users[0]
du = dict_data[user]
du.keys()

dict_keys(['train', 'candi'])

In [7]:
# user history to create input prompt (user_info)
print(du["train"])

{1: {'title': 'Bourne Ultimatum', 'categories': 'Studio Specials, Universal Studios Home Entertainment, All Universal Studios Titles', 'description': '', 'rating': 5.0, 'review': 'awesome movie', 'summary': '**Item Summary: "Bourne Ultimatum"**\n\n- **Title:** Bourne Ultimatum\n- **Categories:** This item falls under several categories, including Studio Specials, Universal Studios Home Entertainment, and All Universal Studios Titles. This classification highlights its association with a prominent studio known for quality entertainment.\n- **Description:** Currently, there is no detailed description provided for the item.\n\n"Bourne Ultimatum" is a notable film that is part of the acclaimed Bourne series, recognized for its thrilling action and intricate plot. As part of Universal Studios\' collection, it reflects the studio\'s commitment to delivering high-quality cinematic experiences.'}, 2: {'title': "TYLER PERRY'S A MADEA CHRISTMAS", 'categories': 'Studio Specials, Lionsgate Home En

In [8]:
# candidate items where flag=1 means the user's latest item and flag=0 means that the user did not interact
# this information is inputted to prompt by removing flag information
print(du["candi"])

{1: {'title': 'Seven Year Itch VHS', 'categories': 'Genre for Featured Categories, Comedy', 'description': '', 'summary': 'The item is a VHS tape of the classic film "Seven Year Itch," a renowned comedy that explores themes of love and infidelity. It falls under the featured categories of comedy, making it an ideal choice for fans of humorous storytelling. While the description lacks additional details, the title itself evokes nostalgia for vintage film formats and classic cinema. This VHS edition is perfect for collectors or enthusiasts looking to enjoy a timeless comedic experience.', 'flag': 0}, 2: {'title': 'Chicago', 'categories': 'Studio Specials, Miramax Home Entertainment, All Titles', 'description': 'Earning 13 Academy Award nominations including Best Picture, Best Director, Best Actress, and Best Supporting Actress, Chicago stars Catherine Zeta-Jones, Renee Zellweger and Richard Gere. Directed by Rob Marshall, it&#8217;s a dazzling spectacle that was cheered by both audiences

In [9]:
# Baseline prompt
type_prompt = "ItemAll_MethodBaseline"  # old name; prompt name for submitted paper is changed to ``Baseline'' simply.
configs = dl.load_configs(data_name=data_name, type_prompt=type_prompt)

from src.prompt import Prompt
p = Prompt(llm, configs)
messages = p.transform(du)
print(messages[0]["content"])  # system prompt
print("==" * 50)
print(messages[1]["content"])  # user prompt

You are an AI assistant that helps people find information.
# Requirements:
you must rank candidate products that will be provided below to the target user for recommendation.

# Observation:
{1: {'title': 'Bourne Ultimatum', 'categories': 'Studio Specials, Universal Studios Home Entertainment, All Universal Studios Titles', 'description': '', 'rating': 5.0, 'review': 'awesome movie'}, 2: {'title': "TYLER PERRY'S A MADEA CHRISTMAS", 'categories': 'Studio Specials, Lionsgate Home Entertainment, All Lionsgate Titles', 'description': "Madea gets coaxed into helping a friend pay her daughter a surprise visit in the country for Christmas, but the biggest surprise is what they'll find when they arrive. As the small, rural town prepares for its annual Christmas Jubilee, new secrets are revealed and old relationships are tested while Madea dishes her own brand of Christmas Spirit to all.", 'rating': 5.0, 'review': 'funny movie'}, 3: {'title': 'The Bourne Supremacy', 'categories': 'Blu-ray, Mov

In [10]:
# output
output, dict_log = llm(messages, log=True)
output, dict_log["time"], dict_log["input token"], dict_log["output token"]

('[4,2,1,8,9,3,10,5,6,7]', 0.932244062423706, 1837, 22)

In [11]:
# computing nDCG
from src import post_process
d_ = post_process.predict(p.flag, llm, messages)
pd.Series(d_)

flag             [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
pred            [8, 9, 5, 10, 3, 2, 1, 7, 6, 4]
input token                                1837
output token                                 22
time                                   0.931067
score                                       1.0
dtype: object

In [12]:
# prompts in ection 4
types_prompt_sec4 = [
    f"ItemAll_Method{b}" 
    for b in [
        "Baseline", 
        "Emotion", 
        "Re-Reading",
        "Baseline_SystemRole",  # Role
        "Baseline_Naming",  # Naming
        "Bothinst",  # Both-Inst
        "RecencyFocused",  # Recency-Focused
        "Pretend",
        "Baseline_ItemSummary",  # Summarize-Item
        "StepBack",  # Step-Back
        "ReAct",
        "Rephrase",
        "Echo",
        "UserSummarization",  # Summarize-User
        "ItemGenerate",  # Generate-Item
        "ItemGenerateTrue",  # Reuse-Item
        "Explain",
        "Mock",                
        "ZSCoT",  # Step-by-Step
        "TakeBreath",  # Deep-Breath
        "PlanSolve"  # Plan-Solve
    ]
]


# prompts in section 5.1
L = ["Rephrase", "StepBack", "ReAct"]
T = ["Rephrase_ReAct", "Rephrase_StepBack"]
for a in L:
    t = f"SelfRefine_{a}"
    T.append(t)    
for a in L:
    t = f"SelfConsistency_{a}"
    T.append(t)    

types_prompt_sec5 = [
    f"ItemAll_Method{b}" for b in ["Baseline"] + L + [f"Combo_{t}" for t in T]
]

In [13]:
type_prompt = types_prompt_sec4[11]
print("using prompt is : ", type_prompt)
print("- " * 50)
configs = dl.load_configs(data_name=data_name, type_prompt=type_prompt)

from src.prompt import Prompt
p = Prompt(llm, configs)
messages = p.transform(du)

print(messages[0]["content"])  # system prompt
print("==" * 50)
print(messages[1]["content"])  # user prompt
print("==" * 50)
print(messages[2]["content"])  # assistant prompt
print("==" * 50)
print(messages[3]["content"])  # user prompt

using prompt is :  ItemAll_MethodRephrase
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
You are an AI assistant that helps people find information.
# Requirements:
you must rank candidate products that will be provided below to the target user for recommendation.

# Observation:
{1: {'title': 'Bourne Ultimatum', 'categories': 'Studio Specials, Universal Studios Home Entertainment, All Universal Studios Titles', 'description': '', 'rating': 5.0, 'review': 'awesome movie'}, 2: {'title': "TYLER PERRY'S A MADEA CHRISTMAS", 'categories': 'Studio Specials, Lionsgate Home Entertainment, All Lionsgate Titles', 'description': "Madea gets coaxed into helping a friend pay her daughter a surprise visit in the country for Christmas, but the biggest surprise is what they'll find when they arrive. As the small, rural town prepares for its annual Christmas Jubilee, new secrets are revealed and old relationships are tested while Madea dishes her ow

In [14]:
# nDCG
from src import post_process
d_ = post_process.predict(p.flag, llm, messages)
pd.Series(d_)

flag             [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
pred            [5, 7, 6, 10, 2, 3, 1, 8, 9, 4]
input token                                2234
output token                                 31
time                                   1.003135
score                                       1.0
dtype: object

In [15]:
type_prompt = types_prompt_sec4[10]
print("using prompt is : ", type_prompt)
print("- " * 50)
configs = dl.load_configs(data_name=data_name, type_prompt=type_prompt)

from src.prompt import Prompt
p = Prompt(llm, configs)
messages = p.transform(du)
print(messages[0]["content"])  # system prompt
print("==" * 50)
print(messages[1]["content"])  # user prompt
print("==" * 50)
print(messages[2]["content"])  # assistant prompt
print("==" * 50)
print(messages[3]["content"])  # user prompt

using prompt is :  ItemAll_MethodReAct
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
You are an AI assistant that helps people find information.
# Requirements:
you must rank candidate products that will be provided below to the target user for recommendation.

# Observation:
{1: {'title': 'Bourne Ultimatum', 'categories': 'Studio Specials, Universal Studios Home Entertainment, All Universal Studios Titles', 'description': '', 'rating': 5.0, 'review': 'awesome movie'}, 2: {'title': "TYLER PERRY'S A MADEA CHRISTMAS", 'categories': 'Studio Specials, Lionsgate Home Entertainment, All Lionsgate Titles', 'description': "Madea gets coaxed into helping a friend pay her daughter a surprise visit in the country for Christmas, but the biggest surprise is what they'll find when they arrive. As the small, rural town prepares for its annual Christmas Jubilee, new secrets are revealed and old relationships are tested while Madea dishes her own b

In [16]:
type_prompt = types_prompt_sec4[9]
print("using prompt is : ", type_prompt)
print("- " * 50)
configs = dl.load_configs(data_name=data_name, type_prompt=type_prompt)

from src.prompt import Prompt
p = Prompt(llm, configs)
messages = p.transform(du)
print(messages[0]["content"])  # system prompt
print("==" * 50)
print(messages[1]["content"])  # user prompt
print("==" * 50)
print(messages[2]["content"])  # assistant prompt
print("==" * 50)
print(messages[3]["content"])  # user prompt

using prompt is :  ItemAll_MethodStepBack
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
You are an AI assistant that helps people find information.
# Requirements:
you must rank candidate products that will be provided below to the target user for recommendation.

# Observation:
{1: {'title': 'Bourne Ultimatum', 'categories': 'Studio Specials, Universal Studios Home Entertainment, All Universal Studios Titles', 'description': '', 'rating': 5.0, 'review': 'awesome movie'}, 2: {'title': "TYLER PERRY'S A MADEA CHRISTMAS", 'categories': 'Studio Specials, Lionsgate Home Entertainment, All Lionsgate Titles', 'description': "Madea gets coaxed into helping a friend pay her daughter a surprise visit in the country for Christmas, but the biggest surprise is what they'll find when they arrive. As the small, rural town prepares for its annual Christmas Jubilee, new secrets are revealed and old relationships are tested while Madea dishes her ow

In [17]:
type_prompt = types_prompt_sec4[18]
print("using prompt is : ", type_prompt)
print("- " * 50)
configs = dl.load_configs(data_name=data_name, type_prompt=type_prompt)

from src.prompt import Prompt
p = Prompt(llm, configs)
messages = p.transform(du)
print(messages[0]["content"])  # system prompt
print("==" * 50)
print(messages[1]["content"])  # user prompt
print("==" * 50)
print(messages[2]["content"])  # assistant prompt
print("==" * 50)
print(messages[3]["content"])  # user prompt
print("==" * 50)
print(messages[4]["content"])  # assistant prompt
print("==" * 50)
print(messages[5]["content"])  # user prompt

using prompt is :  ItemAll_MethodZSCoT
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
You are an AI assistant that helps people find information.
# Requirements:
you must rank candidate products that will be provided below to the target user for recommendation.

# Observation:
{1: {'title': 'Bourne Ultimatum', 'categories': 'Studio Specials, Universal Studios Home Entertainment, All Universal Studios Titles', 'description': '', 'rating': 5.0, 'review': 'awesome movie'}, 2: {'title': "TYLER PERRY'S A MADEA CHRISTMAS", 'categories': 'Studio Specials, Lionsgate Home Entertainment, All Lionsgate Titles', 'description': "Madea gets coaxed into helping a friend pay her daughter a surprise visit in the country for Christmas, but the biggest surprise is what they'll find when they arrive. As the small, rural town prepares for its annual Christmas Jubilee, new secrets are revealed and old relationships are tested while Madea dishes her own b

In [18]:
type_prompt = types_prompt_sec4[11]
print("using prompt is : ", type_prompt)
print("- " * 50)
configs = dl.load_configs(data_name=data_name, type_prompt=type_prompt)

from src.prompt import Prompt
p = Prompt(llm, configs)
messages = p.transform(du)

print(messages[0]["content"])  # system prompt
print("==" * 50)
print(messages[1]["content"])  # user prompt
print("==" * 50)
print(messages[2]["content"])  # assistant prompt
print("==" * 50)
print(messages[3]["content"])  # user prompt

using prompt is :  ItemAll_MethodRephrase
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
You are an AI assistant that helps people find information.
# Requirements:
you must rank candidate products that will be provided below to the target user for recommendation.

# Observation:
{1: {'title': 'Bourne Ultimatum', 'categories': 'Studio Specials, Universal Studios Home Entertainment, All Universal Studios Titles', 'description': '', 'rating': 5.0, 'review': 'awesome movie'}, 2: {'title': "TYLER PERRY'S A MADEA CHRISTMAS", 'categories': 'Studio Specials, Lionsgate Home Entertainment, All Lionsgate Titles', 'description': "Madea gets coaxed into helping a friend pay her daughter a surprise visit in the country for Christmas, but the biggest surprise is what they'll find when they arrive. As the small, rural town prepares for its annual Christmas Jubilee, new secrets are revealed and old relationships are tested while Madea dishes her ow

In [19]:
type_prompt = types_prompt_sec4[5]
print("using prompt is : ", type_prompt)
print("- " * 50)
configs = dl.load_configs(data_name=data_name, type_prompt=type_prompt)

from src.prompt import Prompt
p = Prompt(llm, configs)
messages = p.transform(du)

print(messages[0]["content"])  # system prompt
print("==" * 50)
print(messages[1]["content"])  # user prompt

using prompt is :  ItemAll_MethodBothinst
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
You are an AI assistant that helps people find information.
# Requirements:
you must rank candidate products that will be provided below to the target user for recommendation.

# Observation:
{1: {'title': 'Bourne Ultimatum', 'categories': 'Studio Specials, Universal Studios Home Entertainment, All Universal Studios Titles', 'description': '', 'rating': 5.0, 'review': 'awesome movie'}, 2: {'title': "TYLER PERRY'S A MADEA CHRISTMAS", 'categories': 'Studio Specials, Lionsgate Home Entertainment, All Lionsgate Titles', 'description': "Madea gets coaxed into helping a friend pay her daughter a surprise visit in the country for Christmas, but the biggest surprise is what they'll find when they arrive. As the small, rural town prepares for its annual Christmas Jubilee, new secrets are revealed and old relationships are tested while Madea dishes her ow

In [20]:
type_prompt = types_prompt_sec4[7]
print("using prompt is : ", type_prompt)
print("- " * 50)
configs = dl.load_configs(data_name=data_name, type_prompt=type_prompt)

from src.prompt import Prompt
p = Prompt(llm, configs)
messages = p.transform(du)

print(messages[0]["content"])  # system prompt
print("==" * 50)
print(messages[1]["content"])  # user prompt

using prompt is :  ItemAll_MethodPretend
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
You are an AI assistant that pretends to be a person who has interacted with the following items.
# Logs:
{1: {'title': 'Bourne Ultimatum', 'categories': 'Studio Specials, Universal Studios Home Entertainment, All Universal Studios Titles', 'description': '', 'rating': 5.0, 'review': 'awesome movie'}, 2: {'title': "TYLER PERRY'S A MADEA CHRISTMAS", 'categories': 'Studio Specials, Lionsgate Home Entertainment, All Lionsgate Titles', 'description': "Madea gets coaxed into helping a friend pay her daughter a surprise visit in the country for Christmas, but the biggest surprise is what they'll find when they arrive. As the small, rural town prepares for its annual Christmas Jubilee, new secrets are revealed and old relationships are tested while Madea dishes her own brand of Christmas Spirit to all.", 'rating': 5.0, 'review': 'funny movie'}, 3: {'tit

In [21]:
type_prompt = types_prompt_sec4[8]
print("using prompt is : ", type_prompt)
print("- " * 50)
configs = dl.load_configs(data_name=data_name, type_prompt=type_prompt)

from src.prompt import Prompt
p = Prompt(llm, configs)
messages = p.transform(du)

print(messages[0]["content"])  # system prompt
print("==" * 50)
print(messages[1]["content"])  # user prompt

using prompt is :  ItemAll_MethodBaseline_ItemSummary
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
You are an AI assistant that helps people find information.
# Requirements:
you must rank candidate products that will be provided below to the target user for recommendation.

# Observation:
{1: {'summary': '**Item Summary: "Bourne Ultimatum"**\n\n- **Title:** Bourne Ultimatum\n- **Categories:** This item falls under several categories, including Studio Specials, Universal Studios Home Entertainment, and All Universal Studios Titles. This classification highlights its association with a prominent studio known for quality entertainment.\n- **Description:** Currently, there is no detailed description provided for the item.\n\n"Bourne Ultimatum" is a notable film that is part of the acclaimed Bourne series, recognized for its thrilling action and intricate plot. As part of Universal Studios\' collection, it reflects the studio\'s commi

In [22]:
type_prompt = types_prompt_sec4[14]
print("using prompt is : ", type_prompt)
print("- " * 50)
configs = dl.load_configs(data_name=data_name, type_prompt=type_prompt)

from src.prompt import Prompt
p = Prompt(llm, configs)
messages = p.transform(du)

print(messages[0]["content"])  # system prompt
print("==" * 50)
print(messages[1]["content"])  # user prompt
print("==" * 50)
print(messages[2]["content"])  # assistant prompt
print("==" * 50)
print(messages[3]["content"])  # user prompt

using prompt is :  ItemAll_MethodItemGenerate
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
You are an AI assistant that helps people find information.
# Requirements:
you must rank candidate products that will be provided below to the target user for recommendation.

# Observation:
{1: {'title': 'Bourne Ultimatum', 'categories': 'Studio Specials, Universal Studios Home Entertainment, All Universal Studios Titles', 'description': '', 'rating': 5.0, 'review': 'awesome movie'}, 2: {'title': "TYLER PERRY'S A MADEA CHRISTMAS", 'categories': 'Studio Specials, Lionsgate Home Entertainment, All Lionsgate Titles', 'description': "Madea gets coaxed into helping a friend pay her daughter a surprise visit in the country for Christmas, but the biggest surprise is what they'll find when they arrive. As the small, rural town prepares for its annual Christmas Jubilee, new secrets are revealed and old relationships are tested while Madea dishes he

In [23]:
type_prompt = types_prompt_sec5[4]
print("using prompt is : ", type_prompt)
print("- " * 50)
configs = dl.load_configs(data_name=data_name, type_prompt=type_prompt)

from src.prompt import Prompt
p = Prompt(llm, configs)
messages = p.transform(du)
print(messages[0]["content"])  # system prompt
print("==" * 50)
print(messages[1]["content"])  # user prompt
print("==" * 50)
print(messages[2]["content"])  # assistant prompt
print("==" * 50)
print(messages[3]["content"])  # user prompt
print("==" * 50)
print(messages[4]["content"])  # assistant prompt
print("==" * 50)
print(messages[5]["content"])  # user prompt

using prompt is :  ItemAll_MethodCombo_Rephrase_ReAct
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
You are an AI assistant that helps people find information.
# Requirements:
you must rank candidate products that will be provided below to the target user for recommendation.

# Observation:
{1: {'title': 'Bourne Ultimatum', 'categories': 'Studio Specials, Universal Studios Home Entertainment, All Universal Studios Titles', 'description': '', 'rating': 5.0, 'review': 'awesome movie'}, 2: {'title': "TYLER PERRY'S A MADEA CHRISTMAS", 'categories': 'Studio Specials, Lionsgate Home Entertainment, All Lionsgate Titles', 'description': "Madea gets coaxed into helping a friend pay her daughter a surprise visit in the country for Christmas, but the biggest surprise is what they'll find when they arrive. As the small, rural town prepares for its annual Christmas Jubilee, new secrets are revealed and old relationships are tested while Madea d

In [24]:
type_prompt = types_prompt_sec5[7]
print("using prompt is : ", type_prompt)
print("- " * 50)
configs = dl.load_configs(data_name=data_name, type_prompt=type_prompt)

from src.prompt import Prompt
p = Prompt(llm, configs)
messages = p.transform(du)
print(messages[0]["content"])  # system prompt
print("==" * 50)
print(messages[1]["content"])  # user prompt
print("==" * 50)
print(messages[2]["content"])  # assistant prompt
print("==" * 50)
print(messages[3]["content"])  # user prompt
print("==" * 50)
print(messages[4]["content"])  # assistant prompt
print("==" * 50)
print(messages[5]["content"])  # user prompt

using prompt is :  ItemAll_MethodCombo_SelfRefine_StepBack
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
You are an AI assistant that helps people find information.
# Requirements:
you must rank candidate products that will be provided below to the target user for recommendation.

# Observation:
{1: {'title': 'Bourne Ultimatum', 'categories': 'Studio Specials, Universal Studios Home Entertainment, All Universal Studios Titles', 'description': '', 'rating': 5.0, 'review': 'awesome movie'}, 2: {'title': "TYLER PERRY'S A MADEA CHRISTMAS", 'categories': 'Studio Specials, Lionsgate Home Entertainment, All Lionsgate Titles', 'description': "Madea gets coaxed into helping a friend pay her daughter a surprise visit in the country for Christmas, but the biggest surprise is what they'll find when they arrive. As the small, rural town prepares for its annual Christmas Jubilee, new secrets are revealed and old relationships are tested while Ma